In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np


transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=3, shuffle=False, num_workers=1)  


validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=3, shuffle=False, num_workers=1)  

NUM_EPOCH = 1

for epoch in range(NUM_EPOCH):
    # load training images of the batch size for every iteration
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data
        

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        batch_size = inputs.shape[0]


        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]

        
        # change inputs to matrix 10000*batch_size
        for bat_idx in range(batch_size):
            
            targMat = inputs[bat_idx][0]
            colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')
            
            if(bat_idx == 0):
                batMat = colVec
            else:
                batMat = np.concatenate((batMat, colVec), axis = 1)
        
                
            
                
        
        





    # load validation images of the batch size for every iteration
    for i, data in enumerate(valloader):
        
        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)    
        
    

[[0.59607846 0.6        0.6156863 ]
 [0.6        0.60784316 0.62352943]
 [0.60784316 0.6117647  0.627451  ]
 ...
 [0.8862745  0.7921569  0.6901961 ]
 [0.90588236 0.7647059  0.6666667 ]
 [0.8745098  0.7529412  0.69803923]]
[[0.6627451  0.7058824  0.80784315]
 [0.6627451  0.7137255  0.7882353 ]
 [0.67058825 0.7137255  0.9019608 ]
 ...
 [0.7411765  0.61960787 0.7372549 ]
 [0.74509805 0.5372549  0.7607843 ]
 [0.7411765  0.47843137 0.7764706 ]]
[[0.72156864 0.6313726  0.56078434]
 [0.7176471  0.63529414 0.5411765 ]
 [0.8352941  0.6431373  0.6156863 ]
 ...
 [0.85490197 0.8666667  0.8392157 ]
 [0.80784315 0.8117647  0.78431374]
 [0.7921569  0.8352941  0.76862746]]
[[0.92941177 0.59607846 0.6       ]
 [0.91764706 0.6        0.60784316]
 [0.9647059  0.60784316 0.6117647 ]
 ...
 [0.8627451  0.8862745  0.7921569 ]
 [0.84705883 0.90588236 0.7647059 ]
 [0.85882354 0.87058824 0.7529412 ]]
[[0.6156863  0.6627451  0.7058824 ]
 [0.62352943 0.6627451  0.70980394]
 [0.627451   0.67058825 0.7137255 ]
 ...

[[0.9019608  0.7019608  0.43137255]
 [0.8862745  0.7058824  0.38039216]
 [0.8784314  0.7137255  0.3372549 ]
 ...
 [0.8509804  0.77254903 0.7254902 ]
 [0.8117647  0.77254903 0.7647059 ]
 [0.83137256 0.8509804  0.84313726]]
[[0.5647059  0.54509807 0.5411765 ]
 [0.5764706  0.54901963 0.56078434]
 [0.5647059  0.59607846 0.54901963]
 ...
 [0.84313726 0.88235295 0.25490198]
 [0.8156863  0.8784314  0.25882354]
 [0.85490197 0.89411765 0.23529412]]
[[0.6509804  0.2509804  0.05882353]
 [0.654902   0.24313726 0.05098039]
 [0.6627451  0.3137255  0.04705882]
 ...
 [0.75686276 0.54509807 0.21960784]
 [0.77254903 0.5764706  0.1254902 ]
 [0.8509804  0.5686275  0.13725491]]
[[0.06666667 0.47058824 0.1882353 ]
 [0.0627451  0.43529412 0.18039216]
 [0.0627451  0.40784314 0.1882353 ]
 ...
 [0.5686275  0.5058824  0.4392157 ]
 [0.52156866 0.4862745  0.48235294]
 [0.5176471  0.50980395 0.48235294]]
[[0.17254902 0.16470589 0.39215687]
 [0.15686275 0.16078432 0.39215687]
 [0.13333334 0.16862746 0.39607844]
 ...

[[0.54509807 0.62352943 0.65882355]
 [0.54901963 0.6392157  0.6627451 ]
 [0.5529412  0.5647059  0.67058825]
 ...
 [0.9254902  0.8862745  0.73333335]
 [0.85882354 0.8235294  0.7411765 ]
 [0.8627451  0.7294118  0.7490196 ]]
[[0.6784314  0.627451   0.90588236]
 [0.6862745  0.65882355 0.8901961 ]
 [0.6901961  0.6666667  0.87058824]
 ...
 [0.25882354 0.20784314 0.11372549]
 [0.27450982 0.23529412 0.11764706]
 [0.1882353  0.19215687 0.08235294]]
[[1.         0.3529412  0.22745098]
 [1.         0.30980393 0.21960784]
 [1.         0.28627452 0.24313726]
 ...
 [0.25490198 0.23529412 0.15294118]
 [0.2627451  0.21568628 0.2509804 ]
 [0.23921569 0.20392157 0.1882353 ]]
[[0.83137256 0.6392157  0.22745098]
 [0.84313726 0.6666667  0.26666668]
 [0.8509804  0.70980394 0.29411766]
 ...
 [0.22352941 0.04313726 0.27450982]
 [0.3019608  0.03921569 0.32941177]
 [0.22745098 0.12156863 0.27450982]]
[[0.98039216 0.48235294 0.57254905]
 [0.9843137  0.4745098  0.5764706 ]
 [0.9843137  0.47058824 0.57254905]
 ...

[[0.7529412  0.88235295 0.98039216]
 [0.87058824 0.827451   0.96862745]
 [0.8745098  0.85490197 0.92941177]
 ...
 [0.4627451  0.36862746 0.38431373]
 [0.37254903 0.47843137 0.38039216]
 [0.40784314 0.5176471  0.3882353 ]]
[[0.98039216 0.9607843  0.9411765 ]
 [0.9490196  0.92941177 0.88235295]
 [0.95686275 0.9098039  0.8862745 ]
 ...
 [0.47058824 0.6156863  0.47058824]
 [0.40784314 0.47843137 0.49019608]
 [0.31764707 0.3372549  0.49411765]]
[[1.         1.         0.99607843]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 ...
 [0.3882353  0.27450982 0.22745098]
 [0.41568628 0.24705882 0.23529412]
 [0.41960785 0.3254902  0.23529412]]
[[0.9843137  0.9764706  0.89411765]
 [0.99215686 0.79607844 0.85882354]
 [0.98039216 0.6784314  0.88235295]
 ...
 [0.39607844 0.34117648 0.2509804 ]
 [0.4        0.28235295 0.21568628]
 [0.45490196 0.2627451  0.17254902]]
[[0.10980392 0.32941177 0.64705884]
 [0.10588235 0.36078432 0.65882355]
 [0.09803922 0.3529412  0.7019608 ]
 ...

[[0.85882354 0.8627451  0.654902  ]
 [0.78431374 0.9254902  0.63529414]
 [0.7882353  0.9843137  0.40784314]
 ...
 [0.20392157 0.21568628 0.3372549 ]
 [0.16862746 0.28235295 0.5529412 ]
 [0.18039216 0.24313726 0.5372549 ]]
[[0.50980395 0.4392157  0.40784314]
 [0.49411765 0.4627451  0.4       ]
 [0.47058824 0.5568628  0.3764706 ]
 ...
 [0.10980392 0.28235295 0.2509804 ]
 [0.07450981 0.10980392 0.23529412]
 [0.0627451  0.09019608 0.11372549]]
[[0.28235295 0.36862746 0.28627452]
 [0.28627452 0.3882353  0.23529412]
 [0.42352942 0.26666668 0.22745098]
 ...
 [0.09803922 0.32156864 0.09411765]
 [0.1254902  0.26666668 0.16470589]
 [0.16078432 0.23529412 0.16078432]]
[[0.4117647  0.4509804  0.43529412]
 [0.41568628 0.48235294 0.4627451 ]
 [0.43137255 0.42352942 0.4392157 ]
 ...
 [0.20392157 0.42745098 0.34509805]
 [0.22352941 0.39607844 0.36862746]
 [0.21568628 0.32156864 0.29411766]]
[[0.5568628  0.41960785 0.18039216]
 [0.54509807 0.40784314 0.13725491]
 [0.50980395 0.3137255  0.1254902 ]
 ...

[[0.90588236 0.92941177 1.        ]
 [0.90588236 0.9764706  0.99607843]
 [0.9098039  1.         0.9607843 ]
 ...
 [0.8117647  0.8509804  0.75686276]
 [0.75686276 0.85882354 0.8352941 ]
 [0.8039216  0.8666667  0.9137255 ]]
[[1.         0.32941177 0.38039216]
 [0.8745098  0.34117648 0.39215687]
 [0.6        0.27450982 0.34509805]
 ...
 [0.84313726 0.85490197 0.8745098 ]
 [0.8745098  0.88235295 0.80784315]
 [0.8784314  0.8509804  0.83137256]]
[[0.4627451  0.44705883 0.41960785]
 [0.4509804  0.42352942 0.43137255]
 [0.45882353 0.4117647  0.42352942]
 ...
 [0.88235295 0.8784314  0.8862745 ]
 [0.76862746 0.8509804  0.9019608 ]
 [0.7607843  0.8627451  0.8862745 ]]
[[0.45882353 0.53333336 0.54901963]
 [0.44313726 0.49411765 0.61960787]
 [0.4627451  0.5686275  0.6117647 ]
 ...
 [0.8627451  0.8901961  0.8627451 ]
 [0.8980392  0.90588236 0.8784314 ]
 [0.8980392  0.8901961  0.9098039 ]]
[[0.6039216  0.5764706  0.7490196 ]
 [0.5764706  0.6117647  0.64705884]
 [0.6156863  0.6        0.5764706 ]
 ...

In [11]:
a = np.array([1,2,3])[np.newaxis]
b = np.array([4,5,6])[np.newaxis]
c = np.array([7,8,9])[np.newaxis]
a = a.T
b = b.T
c = c.T

x = a



print(x)
x = np.concatenate((x,b),axis=1)
print(x)
x = np.concatenate((x,c),axis=1)
print(x)
ones = np.ones((1,3), dtype=int)
print(ones)
x = np.concatenate((x,ones))
print(x)

'''
x = np.column_stack((a,b))
print(x)

x = np.column_stack((x,c))
print(c)
'''







[[1]
 [2]
 [3]]
[[1 4]
 [2 5]
 [3 6]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 1 1]]
[[1 4 7]
 [2 5 8]
 [3 6 9]
 [1 1 1]]


'\nx = np.column_stack((a,b))\nprint(x)\n\nx = np.column_stack((x,c))\nprint(c)\n'

In [25]:
t = np.array([[1,2,3,4],
             [4,5,6,6],
             [7,8,9,10]])

print(t)
print(t.shape)
print(np.product(t.shape))
b = np.reshape(t, (np.product(t.shape), 1), 'F')
print(b)



'''
t = t.T


b = np.reshape(t, (1 ,np.product(t.shape))).T
print(b)
'''

[[ 1  2  3  4]
 [ 4  5  6  6]
 [ 7  8  9 10]]
(3, 4)
12
[[ 1]
 [ 4]
 [ 7]
 [ 2]
 [ 5]
 [ 8]
 [ 3]
 [ 6]
 [ 9]
 [ 4]
 [ 6]
 [10]]


'\nt = t.T\n\n\nb = np.reshape(t, (1 ,np.product(t.shape))).T\nprint(b)\n'

In [ ]:
a = 